<a href="https://colab.research.google.com/github/PenroseTiles/Tensorflow2.0_Tutorials/blob/master/TF2_MNIST_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [0]:
import typing

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model, Input

Testing eager execution, vetarans who worked with TF 1.x will know the pain of debugging without this

In [0]:
a = tf.constant(2.0)
b = tf.constant(3.0)
c = a+b

In [0]:
c.numpy()

5.0

#Load the dataset

In [0]:
mnist = keras.datasets.mnist

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

By using ellipsis, I avoid stating exactly the number of dimensions of the Tensor x_train, I just want to add a dimension to the tensor, regardless of how many dimensions it has

In [0]:
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# Playing with *tf.Dataset*

In [0]:
data = tf.data.Dataset.from_tensor_slices(tf.range(8))
it = iter(data)
print(next(it).numpy())

for i in data:
  print(i.numpy())

0
0
1
2
3
4
5
6
7


Map and Reduce

In [0]:
@tf.function
def sum_data(data):
  return data.reduce(0, lambda state, value : state + value)

@tf.function
def transform_data(data: tf.data.Dataset):
  return data.map(lambda x : x+ 1)

sum_data(data).numpy()
mapped = transform_data(data) #

'''
mapped.numpy() will not work because mapped is an iterable, like every Dataset object
But if I can't call numpy() on Datasets, how do I convert them to non-tensor form? 
You can call as_numpy_iterator() on it, it basically does what we do below: 
'''
for item in mapped:
  print(item.numpy())

1
2
3
4
5
6
7
8


In [0]:
mn = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(2)
# mnist_y = tf.data.Dataset.from_tensor_slices(y_train)
# mnist = tf.data.Dataset.zip((mnist_data_x, mnist_data_y))
# mnist = mnist_x




In [0]:
for image, label in mn:
  print(image.numpy().shape)
  break

(2, 28, 28, 1)


In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)


In [0]:
for images, labels in train_ds:
    print(images.numpy().shape)
    break

(32, 28, 28, 1)


# The model

In [0]:
inputs = Input(shape=(28,28,1))
x = Dense(units=400, activation='relu')(inputs)
y = Dense(units=10)(x)
model = keras.Model(inputs, y, name='mnist_nielsen')
model.summary()

Model: "mnist_nielsen"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
dense_2 (Dense)              (None, 28, 28, 400)       800       
_________________________________________________________________
dense_3 (Dense)              (None, 28, 28, 10)        4010      
Total params: 4,810
Trainable params: 4,810
Non-trainable params: 0
_________________________________________________________________


In [0]:
@tf.function
def train():
  for x,y in dataset:
    with tf.GradientTape() as tape:
      prediction = model(x, training=True)
      loss = loss_fn(prediction,y)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
